In [8]:
import pandas as pd
from qdrant_client.models import PointStruct,VectorParams,Distance 
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
 

In [9]:
df = pd.read_csv(r"D:\Internship\EnterpriseKnowledgeBased\web scrape expirement\FAQ_Data.csv")
df

,question,answer
0,What is Offshore Banking Account?,Account that can be opened by Residents outsi...
1,Who can open OBU Account?,Person resident outside Bangladesh including ...
2,Can customers open a joint OBU account?,"Yes, customers can open a joint OBU account. ..."
3,What type of account can a customer open?,A customer can open a Fixed Deposit account i...
4,Documents required to open OBU account,Document check list for Individuals Copy of v...
...,...,...
94,What is the SWIFT code of City Bank PLC?,Our SWIFT Code is CIBLBDDH. It is also known ...
95,"If I have IB account in one currency, can I se...","Yes. In this case, cross currency conversion ..."
96,Is there any tax or any duty applicable on the...,As per new Offshore Banking Act passed by Nat...
97,How is City Bank offering such high-interest r...,Offshore banking basically is dependent on fo...


In [10]:
#del df

In [11]:
question_answers = df[["question","answer"]]
df['question_answer'] = texts = [f"{row['question']}  {row['answer']}" for _, row in question_answers.iterrows()]
df

,question,answer,question_answer
0,What is Offshore Banking Account?,Account that can be opened by Residents outsi...,What is Offshore Banking Account? Account th...
1,Who can open OBU Account?,Person resident outside Bangladesh including ...,Who can open OBU Account? Person resident ou...
2,Can customers open a joint OBU account?,"Yes, customers can open a joint OBU account. ...","Can customers open a joint OBU account? Yes,..."
3,What type of account can a customer open?,A customer can open a Fixed Deposit account i...,What type of account can a customer open? A ...
4,Documents required to open OBU account,Document check list for Individuals Copy of v...,Documents required to open OBU account Docum...
...,...,...,...
94,What is the SWIFT code of City Bank PLC?,Our SWIFT Code is CIBLBDDH. It is also known ...,What is the SWIFT code of City Bank PLC? Our...
95,"If I have IB account in one currency, can I se...","Yes. In this case, cross currency conversion ...","If I have IB account in one currency, can I se..."
96,Is there any tax or any duty applicable on the...,As per new Offshore Banking Act passed by Nat...,Is there any tax or any duty applicable on the...
97,How is City Bank offering such high-interest r...,Offshore banking basically is dependent on fo...,How is City Bank offering such high-interest r...


In [24]:
# Load the model
model_name = "all-mpnet-base-v2"
embedding_model = SentenceTransformer(model_name)



# Generate embeddings for the texts
def prepare_data(row):
    return {
        "id": int(row.name),  # Using the DataFrame's index as the ID
        "payload": {
            "question": row['question'],
            "answer": row['answer']
        },
        "vector": embedding_model.encode(row['question_answer']).tolist(),  # Encode answers to vectors
    }

# Apply the function to the DataFrame
data_to_insert = df.apply(prepare_data, axis=1).tolist()
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


,question,answer,question_answer
0,What is Offshore Banking Account?,Account that can be opened by Residents outsi...,What is Offshore Banking Account? Account th...
1,Who can open OBU Account?,Person resident outside Bangladesh including ...,Who can open OBU Account? Person resident ou...
2,Can customers open a joint OBU account?,"Yes, customers can open a joint OBU account. ...","Can customers open a joint OBU account? Yes,..."
3,What type of account can a customer open?,A customer can open a Fixed Deposit account i...,What type of account can a customer open? A ...
4,Documents required to open OBU account,Document check list for Individuals Copy of v...,Documents required to open OBU account Docum...
...,...,...,...
94,What is the SWIFT code of City Bank PLC?,Our SWIFT Code is CIBLBDDH. It is also known ...,What is the SWIFT code of City Bank PLC? Our...
95,"If I have IB account in one currency, can I se...","Yes. In this case, cross currency conversion ...","If I have IB account in one currency, can I se..."
96,Is there any tax or any duty applicable on the...,As per new Offshore Banking Act passed by Nat...,Is there any tax or any duty applicable on the...
97,How is City Bank offering such high-interest r...,Offshore banking basically is dependent on fo...,How is City Bank offering such high-interest r...


In [15]:
#inserting data into qdrant collection 

# Connect to Qdrant
client = QdrantClient(host='localhost', port=6333)

# Choose your collection name
collection_name = 'city_bank_faq_data'

# Create collection if it doesn't exist
if collection_name not in client.get_collections().collections:
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            'size':768 ,  # Vector dimension
            'distance': 'Cosine'  # Distance metric for vector comparison
        }
    )

# Insert data
points = [
    PointStruct(
        id=data['id'],
        vector=data['vector'],
        payload=data['payload']
    ) for data in data_to_insert
]

# Upsert points into the collection
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
#search query 
def search_query(query:str):
    query_vector = embedding_model.encode(query)
    search_results = client.search(
        collection_name = collection_name,
        query_vector = query_vector,
        query_filter=None,
        limit = 5
    )
    return search_results

In [ ]:
query = "What is Offshore Banking Account?"
query2 = "What information is required to send remittances to any City Bank account from abroad?"
results = search_query(query2)
results

[ScoredPoint(id=36, version=0, score=0.8223108, payload={'answer': ' To send remittances, please use below information for beneficiary- SWIFT Code of City Bank\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 : CIBLBDDH Bank Name\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 : City Bank PLC Customer’s OBU Account Number\xa0\xa0\xa0\xa0: ………………. Account Name\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 : ……………………', 'question': 'What information is required to send remittances to any City Bank account from abroad?'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=89, version=0, score=0.81574214, payload={'answer': ' To send remittances, please use below information for beneficiary- SWIFT Code of City Bank\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 

## Creating a retreival chain 


In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores.qdrant import Qdrant
from langchain_community.llms import CTransformers
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline


llm = CTransformers(
    model='D:/Internship/EnterpriseKnowledgeBased/model/llama-2-7b-chat.ggmlv3.q4_0.bin',
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8}
)
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {input}

Only return the helpful answer. Answer must be detailed and well explained.
Helpful answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'input'])

city_bank_faq_db_retriever = Qdrant(client=client, embeddings=embedding_model, collection_name="city_bank_faq_data")
retriever = city_bank_faq_db_retriever.as_retriever()

combine_docs_chain = create_stuff_documents_chain(
    llm=llm,
    # Default prompt is loaded from the hub, but we can also modify it
    prompt=prompt,
    # prompt=hub.pull("langchain-ai/retrieval-qa-chat")
)
retrieval_qa_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=combine_docs_chain,
)
response = retrieval_qa_chain.invoke({"input": "What is Offshore Banking Account?"})

d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(
d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\langchain_community\vectorstores\qdrant.py:154: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(


TypeError: string indices must be integers

In [22]:
from langchain import VectorDBQA
qa = VectorDBQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    vectorstore=city_bank_faq_db_retriever,
    return_source_documents=False,
)
qa.run("What is Offshore Banking Account?")

d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\langchain\chains\retrieval_qa\base.py:290: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(
d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


TypeError: string indices must be integers

## Chroma db

In [27]:
df

,question,answer,question_answer
0,What is Offshore Banking Account?,Account that can be opened by Residents outsi...,What is Offshore Banking Account? Account th...
1,Who can open OBU Account?,Person resident outside Bangladesh including ...,Who can open OBU Account? Person resident ou...
2,Can customers open a joint OBU account?,"Yes, customers can open a joint OBU account. ...","Can customers open a joint OBU account? Yes,..."
3,What type of account can a customer open?,A customer can open a Fixed Deposit account i...,What type of account can a customer open? A ...
4,Documents required to open OBU account,Document check list for Individuals Copy of v...,Documents required to open OBU account Docum...
...,...,...,...
94,What is the SWIFT code of City Bank PLC?,Our SWIFT Code is CIBLBDDH. It is also known ...,What is the SWIFT code of City Bank PLC? Our...
95,"If I have IB account in one currency, can I se...","Yes. In this case, cross currency conversion ...","If I have IB account in one currency, can I se..."
96,Is there any tax or any duty applicable on the...,As per new Offshore Banking Act passed by Nat...,Is there any tax or any duty applicable on the...
97,How is City Bank offering such high-interest r...,Offshore banking basically is dependent on fo...,How is City Bank offering such high-interest r...


In [25]:
texts

['What is Offshore Banking Account?   Account that can be opened by Residents outside Bangladesh in foreign currency. For example: Non-resident Bangladeshi nationals (NRBs), Persons of Bangladeshi origin (Bangladeshi holding foreign citizenship), Foreign nationals, Companies/Firms registered and operating abroad, Foreign institutional investors etc. can open this OBU account.',
 'Who can open OBU Account?   Person resident outside Bangladesh including NRBs Foreign nationals Persons of Bangladeshi origin (Bangladeshi holding foreign citizenship) Companies/Firms registered and operating abroad Foreign institutional investors Type A units in EPZs (Export Processing Zones, Private EPZs, Economic Zones and Hi-tech parks) in Bangladesh Note: All individuals will have to be of age 18 & above.',
 'Can customers open a joint OBU account?   Yes, customers can open a joint OBU account. But in such cases both the account holders must individually qualify to open the account as per question 1 or 2 

In [28]:
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
import pandas as pd

# Initialize the directory and model
persist_directory = 'db'
model_name = "all-mpnet-base-v2"
embedding_model = SentenceTransformer(model_name)

# Assuming df is your DataFrame
docs = []
metadatas = []
ids = []
embeddings = []

for index, row in df.iterrows():
    output_str = ""
    # Treat each row as a separate chunk
    for col in df.columns:
        output_str += f"{col}: {row[col]},\n"
    
    # Create embeddings using Sentence Transformer
    embedding = embedding_model.encode(output_str)
    embeddings.append(embedding)
    
    docs.append(output_str)
    metadatas.append({"source": "city bank faq"})
    ids.append(f"id{index}")

# Assuming the rest of your code integrates with Chroma as necessary
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embedding_model.encode)

# Adding the data to the vector store
vector_store.add_texts(docs, metadatas, ids, embeddings)

# Persist the vector store if needed
vector_store.persist()


In [ ]:
vectordb.persist()
vectordb = None